In [ ]:
import sys
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
from collections import defaultdict

repo_root = Path.cwd().parent.parent.parent
sys.path.insert(0, str(repo_root))

from modules._import_helper import safe_import_from

TrainingConfig = safe_import_from('06_deep_learning_systems.src.config', 'TrainingConfig')
SimpleMLP = safe_import_from('06_deep_learning_systems.src.models', 'SimpleMLP')
get_mnist_loaders = safe_import_from('06_deep_learning_systems.src.datasets', 'get_mnist_loaders')
Trainer = safe_import_from('06_deep_learning_systems.src.trainer', 'Trainer')
set_seed = safe_import_from('00_repo_standards.src.mlphys_core.seeding', 'set_seed')

print("✓ Imports successful")

# Setup
reports_dir = Path("../reports/notebook_03")
reports_dir.mkdir(parents=True, exist_ok=True)
set_seed(42)

## Part 1: Optimizer Comparison - SGD vs Adam

### Theory (Minimal)

**SGD (Stochastic Gradient Descent)**:
```
θ_{t+1} = θ_t - lr * ∇L(θ_t)
```
- Simple, stable, requires careful LR tuning
- With momentum: smooth out gradients

**Adam (Adaptive Moment Estimation)**:
```
m_t = β₁*m_{t-1} + (1-β₁)*∇L     # momentum
v_t = β₂*v_{t-1} + (1-β₂)*∇L²    # adaptive LR
θ_{t+1} = θ_t - lr * m_t/√(v_t)
```
- Adapts LR per parameter
- More forgiving to LR choice
- May generalize worse on some problems

Let's compare them!

In [ ]:
# Load data (small subset for speed)
print("Loading MNIST...")
train_loader, val_loader, test_loader = get_mnist_loaders(
    data_dir=Path("../../../data"),
    batch_size=128,
    val_split=0.1,
    num_workers=0,
    seed=42,
)
print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

In [ ]:
def run_training_experiment(optimizer_name: str, lr: float, epochs: int = 5):
    """Run training with specified optimizer and LR."""
    set_seed(42)  # Same init for fair comparison
    
    config = TrainingConfig(
        name=f"{optimizer_name}_lr{lr}",
        seed=42,
        model_type="SimpleMLP",
        input_dim=784,
        hidden_dims=[128, 64],
        output_dim=10,
        batch_size=128,
        num_epochs=epochs,
        learning_rate=lr,
        optimizer=optimizer_name,
        early_stop_patience=100,  # No early stopping
        save_artifacts=False,
        device="cpu",
    )
    
    model = SimpleMLP(
        input_dim=config.input_dim,
        hidden_dims=config.hidden_dims,
        output_dim=config.output_dim,
    )
    
    trainer = Trainer(config, model, device="cpu")
    history = trainer.train(train_loader, val_loader)
    test_metrics = trainer.evaluate(test_loader)
    
    return history, test_metrics

# Experiment 1: SGD with good LR
print("\nExperiment 1: SGD (lr=0.01)")
print("="*60)
history_sgd, metrics_sgd = run_training_experiment("sgd", lr=0.01, epochs=5)

In [ ]:
# Experiment 2: Adam with same LR
print("\nExperiment 2: Adam (lr=0.01)")
print("="*60)
history_adam, metrics_adam = run_training_experiment("adam", lr=0.01, epochs=5)

In [ ]:
# Experiment 3: Adam with smaller LR (more typical)
print("\nExperiment 3: Adam (lr=0.001)")
print("="*60)
history_adam_small, metrics_adam_small = run_training_experiment("adam", lr=0.001, epochs=5)

In [ ]:
# Compare results
print("\n" + "="*60)
print("Comparison Summary")
print("="*60)

results = [
    ("SGD (lr=0.01)", metrics_sgd['accuracy'], history_sgd['train_loss'][-1]),
    ("Adam (lr=0.01)", metrics_adam['accuracy'], history_adam['train_loss'][-1]),
    ("Adam (lr=0.001)", metrics_adam_small['accuracy'], history_adam_small['train_loss'][-1]),
]

for name, acc, train_loss in results:
    print(f"{name:20s} → Test Acc: {acc:.4f}, Train Loss: {train_loss:.4f}")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

epochs = np.arange(1, 6)

# Training loss
axes[0].plot(epochs, history_sgd['train_loss'], 'o-', label='SGD (lr=0.01)', linewidth=2, markersize=8)
axes[0].plot(epochs, history_adam['train_loss'], 's-', label='Adam (lr=0.01)', linewidth=2, markersize=8)
axes[0].plot(epochs, history_adam_small['train_loss'], '^-', label='Adam (lr=0.001)', linewidth=2, markersize=8)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Train Loss', fontsize=12)
axes[0].set_title('Optimizer Comparison: Training Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Validation accuracy
axes[1].plot(epochs, history_sgd['val_accuracy'], 'o-', label='SGD (lr=0.01)', linewidth=2, markersize=8)
axes[1].plot(epochs, history_adam['val_accuracy'], 's-', label='Adam (lr=0.01)', linewidth=2, markersize=8)
axes[1].plot(epochs, history_adam_small['val_accuracy'], '^-', label='Adam (lr=0.001)', linewidth=2, markersize=8)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Validation Accuracy', fontsize=12)
axes[1].set_title('Optimizer Comparison: Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([0.9, 1.0])

plt.tight_layout()
plt.savefig(reports_dir / "optimizer_comparison.png", dpi=150)
plt.show()

print(f"\n✓ Saved to {reports_dir / 'optimizer_comparison.png'}")

### Observations

- **SGD**: Stable but may need more tuning
- **Adam (lr=0.01)**: May be too aggressive (faster but less stable)
- **Adam (lr=0.001)**: Typically good default for Adam

**Rule of thumb**: Start with Adam (lr=1e-3), then try SGD with momentum (lr=1e-2) if you have time to tune.

## Part 2: Learning Rate Sensitivity

**Question**: How sensitive is training to LR choice?

Let's sweep over learning rates.

In [ ]:
# LR sweep for Adam
learning_rates = [1e-4, 3e-4, 1e-3, 3e-3, 1e-2]
lr_results = []

print("Learning Rate Sweep (Adam)")
print("="*60)

for lr in learning_rates:
    print(f"\nTraining with lr={lr:.0e}...")
    history, metrics = run_training_experiment("adam", lr=lr, epochs=5)
    
    lr_results.append({
        'lr': lr,
        'test_acc': metrics['accuracy'],
        'test_loss': metrics['loss'],
        'final_train_loss': history['train_loss'][-1],
        'final_val_acc': history['val_accuracy'][-1],
    })
    
    print(f"  Test acc: {metrics['accuracy']:.4f}")

print("\n" + "="*60)

In [ ]:
# Visualize LR sensitivity
df_lr = pd.DataFrame(lr_results)
print("\nLR Sweep Results:")
print(df_lr.to_string(index=False))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Test accuracy vs LR
axes[0].semilogx(df_lr['lr'], df_lr['test_acc'], 'o-', linewidth=2, markersize=10, color='steelblue')
axes[0].set_xlabel('Learning Rate', fontsize=12)
axes[0].set_ylabel('Test Accuracy', fontsize=12)
axes[0].set_title('LR Sensitivity: Test Accuracy', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].axvline(1e-3, color='red', linestyle='--', alpha=0.5, label='Typical LR')
axes[0].legend()

# Train loss vs LR
axes[1].loglog(df_lr['lr'], df_lr['final_train_loss'], 's-', linewidth=2, markersize=10, color='orange')
axes[1].set_xlabel('Learning Rate', fontsize=12)
axes[1].set_ylabel('Final Train Loss', fontsize=12)
axes[1].set_title('LR Sensitivity: Train Loss', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, which='both')
axes[1].axvline(1e-3, color='red', linestyle='--', alpha=0.5, label='Typical LR')
axes[1].legend()

plt.tight_layout()
plt.savefig(reports_dir / "lr_sensitivity.png", dpi=150)
plt.show()

print(f"\n✓ Saved to {reports_dir / 'lr_sensitivity.png'}")

### Key Insights

- **Too low** (1e-4): Slow convergence, may not reach optimum
- **Sweet spot** (1e-3 to 3e-3): Good balance
- **Too high** (1e-2): May overshoot or be unstable

**Finding good LR**: Look for the "elbow" where accuracy plateaus and loss is low.

## Part 3: Learning Rate Schedules

**Idea**: Start with high LR (fast progress), then decay (fine-tune).

**Common schedules**:
- **StepLR**: Multiply by γ every N epochs
- **CosineAnnealing**: Smooth decay following cosine curve
- **ReduceLROnPlateau**: Decay when val loss plateaus

In [ ]:
def train_with_schedule(schedule_name: str, epochs: int = 10):
    """Train with LR schedule."""
    set_seed(42)
    
    model = SimpleMLP(input_dim=784, hidden_dims=[128, 64], output_dim=10)
    optimizer = optim.Adam(model.parameters(), lr=1e-2)  # Start high
    criterion = nn.CrossEntropyLoss()
    
    # Create scheduler
    if schedule_name == "step":
        scheduler = StepLR(optimizer, step_size=3, gamma=0.5)  # Halve every 3 epochs
    elif schedule_name == "cosine":
        scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
    else:
        scheduler = None
    
    # Training loop
    history = {'train_loss': [], 'val_acc': [], 'lr': []}
    
    for epoch in range(epochs):
        # Train
        model.train()
        total_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        avg_train_loss = total_loss / len(train_loader)
        
        # Validate
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                output = model(X_batch)
                pred = output.argmax(dim=1)
                correct += (pred == y_batch).sum().item()
                total += y_batch.size(0)
        
        val_acc = correct / total
        
        # Record
        current_lr = optimizer.param_groups[0]['lr']
        history['train_loss'].append(avg_train_loss)
        history['val_acc'].append(val_acc)
        history['lr'].append(current_lr)
        
        print(f"Epoch {epoch+1}/{epochs}: loss={avg_train_loss:.4f}, val_acc={val_acc:.4f}, lr={current_lr:.6f}")
        
        # Step scheduler
        if scheduler is not None:
            scheduler.step()
    
    return history

# Experiment: Constant LR
print("\nExperiment: Constant LR (no schedule)")
print("="*60)
history_constant = train_with_schedule("none", epochs=10)

In [ ]:
# Experiment: StepLR
print("\nExperiment: StepLR (halve every 3 epochs)")
print("="*60)
history_step = train_with_schedule("step", epochs=10)

In [ ]:
# Experiment: Cosine Annealing
print("\nExperiment: Cosine Annealing")
print("="*60)
history_cosine = train_with_schedule("cosine", epochs=10)

In [ ]:
# Visualize schedules
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

epochs = np.arange(1, 11)

# LR schedules
axes[0, 0].plot(epochs, history_constant['lr'], 'o-', label='Constant', linewidth=2, markersize=6)
axes[0, 0].plot(epochs, history_step['lr'], 's-', label='StepLR', linewidth=2, markersize=6)
axes[0, 0].plot(epochs, history_cosine['lr'], '^-', label='Cosine', linewidth=2, markersize=6)
axes[0, 0].set_xlabel('Epoch', fontsize=12)
axes[0, 0].set_ylabel('Learning Rate', fontsize=12)
axes[0, 0].set_title('LR Schedules', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# Train loss
axes[0, 1].plot(epochs, history_constant['train_loss'], 'o-', label='Constant', linewidth=2, markersize=6)
axes[0, 1].plot(epochs, history_step['train_loss'], 's-', label='StepLR', linewidth=2, markersize=6)
axes[0, 1].plot(epochs, history_cosine['train_loss'], '^-', label='Cosine', linewidth=2, markersize=6)
axes[0, 1].set_xlabel('Epoch', fontsize=12)
axes[0, 1].set_ylabel('Train Loss', fontsize=12)
axes[0, 1].set_title('Training Loss', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=11)
axes[0, 1].grid(True, alpha=0.3)

# Val accuracy
axes[1, 0].plot(epochs, history_constant['val_acc'], 'o-', label='Constant', linewidth=2, markersize=6)
axes[1, 0].plot(epochs, history_step['val_acc'], 's-', label='StepLR', linewidth=2, markersize=6)
axes[1, 0].plot(epochs, history_cosine['val_acc'], '^-', label='Cosine', linewidth=2, markersize=6)
axes[1, 0].set_xlabel('Epoch', fontsize=12)
axes[1, 0].set_ylabel('Validation Accuracy', fontsize=12)
axes[1, 0].set_title('Validation Accuracy', fontsize=14, fontweight='bold')
axes[1, 0].legend(fontsize=11)
axes[1, 0].grid(True, alpha=0.3)

# Final comparison table
axes[1, 1].axis('off')
table_data = [
    ['Schedule', 'Final Val Acc', 'Final Loss'],
    ['Constant', f"{history_constant['val_acc'][-1]:.4f}", f"{history_constant['train_loss'][-1]:.4f}"],
    ['StepLR', f"{history_step['val_acc'][-1]:.4f}", f"{history_step['train_loss'][-1]:.4f}"],
    ['Cosine', f"{history_cosine['val_acc'][-1]:.4f}", f"{history_cosine['train_loss'][-1]:.4f}"],
]
table = axes[1, 1].table(cellText=table_data, cellLoc='center', loc='center',
                         colWidths=[0.3, 0.35, 0.35])
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 2)
# Style header
for i in range(3):
    table[(0, i)].set_facecolor('#4CAF50')
    table[(0, i)].set_text_props(weight='bold', color='white')

plt.tight_layout()
plt.savefig(reports_dir / "lr_schedules.png", dpi=150)
plt.show()

print(f"\n✓ Saved to {reports_dir / 'lr_schedules.png'}")

### Observations

- **Constant**: Simple, but may overshoot later
- **StepLR**: Sudden drops help escape plateaus
- **Cosine**: Smooth decay, popular for training from scratch

**When to use schedules**: Long training runs (>20 epochs), or when you see val loss plateau.

## Part 4: Regularization - Weight Decay & Dropout

**Goal**: Prevent overfitting

**Weight Decay (L2 regularization)**:
```
Loss = CrossEntropy + λ * ||θ||²
```
Penalizes large weights → smoother decision boundaries

**Dropout**:
Randomly zero out neurons during training → ensemble effect

In [ ]:
# Regularization sweep
reg_configs = [
    {"name": "No regularization", "weight_decay": 0.0, "dropout": 0.0},
    {"name": "Weight decay", "weight_decay": 1e-4, "dropout": 0.0},
    {"name": "Dropout", "weight_decay": 0.0, "dropout": 0.3},
    {"name": "Both", "weight_decay": 1e-4, "dropout": 0.3},
]

reg_results = []

print("Regularization Comparison")
print("="*60)

for cfg in reg_configs:
    print(f"\n{cfg['name']}...")
    set_seed(42)
    
    config = TrainingConfig(
        name=cfg['name'],
        seed=42,
        model_type="SimpleMLP",
        input_dim=784,
        hidden_dims=[128, 64],
        output_dim=10,
        dropout=cfg['dropout'],
        batch_size=128,
        num_epochs=5,
        learning_rate=1e-3,
        weight_decay=cfg['weight_decay'],
        optimizer="adam",
        early_stop_patience=100,
        save_artifacts=False,
        device="cpu",
    )
    
    model = SimpleMLP(
        input_dim=config.input_dim,
        hidden_dims=config.hidden_dims,
        output_dim=config.output_dim,
        dropout=config.dropout,
    )
    
    trainer = Trainer(config, model, device="cpu")
    history = trainer.train(train_loader, val_loader)
    test_metrics = trainer.evaluate(test_loader)
    
    reg_results.append({
        'name': cfg['name'],
        'test_acc': test_metrics['accuracy'],
        'train_loss': history['train_loss'][-1],
        'val_loss': history['val_loss'][-1],
        'gap': history['train_loss'][-1] - history['val_loss'][-1],  # Overfitting indicator
    })
    
    print(f"  Test acc: {test_metrics['accuracy']:.4f}")
    print(f"  Train/Val gap: {reg_results[-1]['gap']:.4f}")

In [ ]:
# Visualize regularization effects
df_reg = pd.DataFrame(reg_results)
print("\nRegularization Results:")
print(df_reg.to_string(index=False))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x_pos = np.arange(len(df_reg))

# Test accuracy
axes[0].bar(x_pos, df_reg['test_acc'], color='steelblue', alpha=0.7)
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(df_reg['name'], rotation=15, ha='right')
axes[0].set_ylabel('Test Accuracy', fontsize=12)
axes[0].set_title('Regularization Effect on Test Accuracy', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_ylim([0.95, 0.98])

# Train-Val gap (overfitting indicator)
colors = ['red' if gap < 0 else 'orange' for gap in df_reg['gap']]
axes[1].bar(x_pos, np.abs(df_reg['gap']), color=colors, alpha=0.7)
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(df_reg['name'], rotation=15, ha='right')
axes[1].set_ylabel('|Train Loss - Val Loss|', fontsize=12)
axes[1].set_title('Overfitting Gap (smaller = better)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(reports_dir / "regularization_comparison.png", dpi=150)
plt.show()

print(f"\n✓ Saved to {reports_dir / 'regularization_comparison.png'}")

### Key Insights

- **No regularization**: May overfit (train loss << val loss)
- **Weight decay**: Smooth regularization, good default (1e-4 to 1e-5)
- **Dropout**: Stronger regularization, use when overfitting is severe
- **Both**: Can combine, but may hurt underfitting regime

**Rule**: Start without regularization on small datasets. Add if train/val gap is large.

## Part 5: Save Summary

In [ ]:
summary = {
    "optimizer_comparison": {
        "sgd_lr0.01": float(metrics_sgd['accuracy']),
        "adam_lr0.01": float(metrics_adam['accuracy']),
        "adam_lr0.001": float(metrics_adam_small['accuracy']),
    },
    "lr_sensitivity": df_lr.to_dict('records'),
    "best_lr": float(df_lr.loc[df_lr['test_acc'].idxmax(), 'lr']),
    "lr_schedules": {
        "constant": float(history_constant['val_acc'][-1]),
        "step": float(history_step['val_acc'][-1]),
        "cosine": float(history_cosine['val_acc'][-1]),
    },
    "regularization": df_reg.to_dict('records'),
    "recommendations": {
        "default_optimizer": "adam",
        "default_lr_adam": 0.001,
        "default_lr_sgd": 0.01,
        "default_weight_decay": 1e-4,
        "use_schedule_if": "training >20 epochs or val loss plateaus",
        "use_dropout_if": "severe overfitting (train_loss << val_loss)",
    }
}

with open(reports_dir / "summary.json", 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✓ Saved summary to {reports_dir / 'summary.json'}")

## Key Takeaways

### ✅ Optimization Best Practices

1. **Start simple**: Adam with lr=1e-3, no regularization
2. **LR search**: Try [1e-4, 3e-4, 1e-3, 3e-3] and pick best
3. **Add schedule**: If training >20 epochs or val loss plateaus
4. **Regularize if overfitting**: Weight decay (1e-4) first, then dropout (0.3)
5. **SGD alternative**: If you have time to tune, SGD+momentum can generalize better

### ⚠️ Common Pitfalls

1. **LR too high**: Loss explodes or oscillates wildly
2. **LR too low**: Training is painfully slow, gets stuck in plateaus
3. **Over-regularization**: Hurts when model is underfitting
4. **Tuning prematurely**: Fix bugs (overfit test) before tuning hyperparameters
5. **Ignoring train/val gap**: Sign of overfitting that regularization can fix

### 🔧 Debugging Optimization Issues

| Symptom | Likely Cause | Fix |
|---------|--------------|-----|
| Loss explodes | LR too high | Reduce by 10x |
| Loss barely moves | LR too low | Increase by 10x |
| Val loss >> train loss | Overfitting | Add weight decay or dropout |
| Both losses high | Underfitting | Remove regularization, increase capacity |
| Validation plateaus | Local minimum | Try LR schedule or restart with different seed |
| Unstable training | Batch size or LR | Increase batch size or reduce LR |

### 📋 Hyperparameter Tuning Workflow

```python
# 1. Start with defaults
optimizer = Adam(lr=1e-3, weight_decay=0)

# 2. Verify overfit test passes
assert overfit_test_passes(), "Fix bugs first!"

# 3. Quick LR search (3-5 values)
for lr in [1e-4, 3e-4, 1e-3, 3e-3]:
    train_and_evaluate(lr)

# 4. Check for overfitting
if train_loss << val_loss:
    add_weight_decay(1e-4)
    # or add_dropout(0.3)

# 5. Add schedule if needed
if num_epochs > 20:
    use_cosine_schedule()
```

## Exercises

### Exercise 1: Find Optimal LR for SGD

Run an LR sweep for SGD (not Adam) and find the best learning rate.

In [ ]:
# Your code here

### Exercise 2: Implement ReduceLROnPlateau

Use `torch.optim.lr_scheduler.ReduceLROnPlateau` to automatically reduce LR when validation loss stops improving.

In [ ]:
# Your code here

### Exercise 3: Overfitting Challenge

Intentionally create a model that overfits severely (train acc >99%, val acc <90%). Then fix it with regularization.

In [ ]:
# Your code here
# Hints: very large model, no regularization, train longer

---

## Solutions

<details>
<summary>Click to reveal solutions</summary>

### Solution 1: SGD LR Sweep

In [ ]:
# SGD LR sweep
sgd_lrs = [1e-3, 3e-3, 1e-2, 3e-2, 1e-1]
sgd_results = []

for lr in sgd_lrs:
    print(f"SGD with lr={lr:.0e}")
    history, metrics = run_training_experiment("sgd", lr=lr, epochs=5)
    sgd_results.append({'lr': lr, 'test_acc': metrics['accuracy']})
    print(f"  Test acc: {metrics['accuracy']:.4f}\n")

df_sgd = pd.DataFrame(sgd_results)
best_lr = df_sgd.loc[df_sgd['test_acc'].idxmax(), 'lr']
print(f"Best SGD LR: {best_lr:.0e}")
print(df_sgd)

### Solution 2: ReduceLROnPlateau

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

set_seed(42)
model = SimpleMLP(784, [128, 64], 10)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    # Train
    model.train()
    for X, y in train_loader:
        optimizer.zero_grad()
        loss = criterion(model(X), y)
        loss.backward()
        optimizer.step()
    
    # Validate
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X, y in val_loader:
            val_loss += criterion(model(X), y).item()
    val_loss /= len(val_loader)
    
    print(f"Epoch {epoch+1}: val_loss={val_loss:.4f}, lr={optimizer.param_groups[0]['lr']:.6f}")
    
    # Step scheduler based on val loss
    scheduler.step(val_loss)

print("\n→ LR reduced automatically when val loss plateaued!")

### Solution 3: Overfitting Challenge

In [ ]:
# Create overfitting scenario
print("Step 1: Create severely overfitting model\n")
set_seed(42)

# Very large model, no regularization
config_overfit = TrainingConfig(
    name="overfit",
    seed=42,
    model_type="SimpleMLP",
    input_dim=784,
    hidden_dims=[512, 512, 256],  # Very large
    output_dim=10,
    dropout=0.0,  # No dropout
    batch_size=128,
    num_epochs=15,  # Train longer
    learning_rate=1e-3,
    weight_decay=0.0,  # No weight decay
    optimizer="adam",
    early_stop_patience=100,
    save_artifacts=False,
    device="cpu",
)

model_overfit = SimpleMLP(
    input_dim=config_overfit.input_dim,
    hidden_dims=config_overfit.hidden_dims,
    output_dim=config_overfit.output_dim,
)

trainer_overfit = Trainer(config_overfit, model_overfit, device="cpu")
history_overfit = trainer_overfit.train(train_loader, val_loader)

print(f"\nOverfitting achieved:")
print(f"  Final train loss: {history_overfit['train_loss'][-1]:.4f}")
print(f"  Final val loss: {history_overfit['val_loss'][-1]:.4f}")
print(f"  Gap: {history_overfit['val_loss'][-1] - history_overfit['train_loss'][-1]:.4f}")

# Fix with regularization
print("\n\nStep 2: Fix with regularization\n")
set_seed(42)

config_fixed = TrainingConfig(
    name="fixed",
    seed=42,
    model_type="SimpleMLP",
    input_dim=784,
    hidden_dims=[512, 512, 256],
    output_dim=10,
    dropout=0.5,  # Added dropout
    batch_size=128,
    num_epochs=15,
    learning_rate=1e-3,
    weight_decay=1e-3,  # Added weight decay
    optimizer="adam",
    early_stop_patience=100,
    save_artifacts=False,
    device="cpu",
)

model_fixed = SimpleMLP(
    input_dim=config_fixed.input_dim,
    hidden_dims=config_fixed.hidden_dims,
    output_dim=config_fixed.output_dim,
    dropout=config_fixed.dropout,
)

trainer_fixed = Trainer(config_fixed, model_fixed, device="cpu")
history_fixed = trainer_fixed.train(train_loader, val_loader)

print(f"\nAfter regularization:")
print(f"  Final train loss: {history_fixed['train_loss'][-1]:.4f}")
print(f"  Final val loss: {history_fixed['val_loss'][-1]:.4f}")
print(f"  Gap: {history_fixed['val_loss'][-1] - history_fixed['train_loss'][-1]:.4f}")
print("\n✓ Overfitting reduced!")

</details>

---

## Next Steps

- **Notebook 04**: Monitoring & error analysis - confusion matrices, per-class metrics, error inspection

**Remember**: Start simple (Adam, lr=1e-3, no regularization), verify it works (overfit test), then tune systematically.